### ספריות


In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import Point
import os
import sys
import math
import warnings
from geopy.geocoders import Nominatim
import pyproj

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

### העלת משתנים להרצת הקוד


In [ ]:
cwd = os.getcwd()

education_folder_path = os.path.dirname(cwd)

sys.path.append(education_folder_path)

### פונקציות גלובליות


In [ ]:
from global_functions import remove_spaces_in_columns, up_load_shp, up_load_df, drop_geo, change_cols_names, get_newest_date_file, get_output_folder_location

### הגדרת מיקום הפלט


In [ ]:
output_folder_location=get_output_folder_location(r'{}\background_files\output_folder_location.txt'.format(cwd))
out_shp = r'{}\education.shp'.format(output_folder_location)

## העלאת טבלאות


In [ ]:
#בתי ספר וגנים מעיריית בית שמש
BShemesh=up_load_df(r'{}\betshemesh_muni\Intermediates'.format(cwd),'BShemesh_final')
BShemesh = remove_spaces_in_columns(BShemesh)

In [ ]:
#בתי ספר וגנים מעיריית ירושלים
JLM=up_load_df(r'{}\jerusalem_muni\Intermediates'.format(cwd),'JLM_final')

In [ ]:
#בתי ספר ממשרד החינוך במרחב
Gschool=up_load_df(r'{}\ministry_of_education\Intermediates'.format(cwd),'Gschool_final')
Gschool = remove_spaces_in_columns(Gschool)